In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np                                
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import keras as k
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import h5py
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data = pd.read_csv('../input/Train.csv')
data.head()

In [ ]:
from scipy.interpolate import griddata
from PIL import Image
def interpolate(df):
    img_x = 32
    img_y = 32

    # A Mesh grid for image
    grid_X, grid_Y = np.meshgrid(np.arange(0, img_x, 1), np.arange(0, img_y, 1))
    
    columns = [ "sparse_"+str(x) for x in range(1024) ]
    images = df[columns].values

    for i in range(len(images)):
        image = images[i].reshape(img_x,img_y)
        notNaN_mask = np.isnan(image) == False
        points = np.argwhere(notNaN_mask)   # Data point coordinates.
        values = image[notNaN_mask]         # Data values.
        #xi = np.argwhere(np.isnan(image))
        function = griddata((points[:,0],points[:,1]), values , (grid_Y, grid_X) , method='cubic',fill_value=255)
        df.iloc[i][columns] = function.flatten()
        #im = Image.fromarray( function )
        #im = im.convert("L")
        #im.save('train/image_'+str(i)+'.png')
interpolate(data)

In [ ]:
columns = [ "sparse_"+str(x) for x in range(0,1024) ]
columns.append('label')
train = pd.DataFrame(data[columns])

In [ ]:
Y = np.array(train['label'])
train = np.array(train.drop('label',1))
from sklearn import preprocessing
train = preprocessing.StandardScaler().fit_transform(train)
x_train = train.reshape(train.shape[0], 32, 32,1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
x_train=x_train.astype("float32")  

In [ ]:
Y.shape

In [ ]:
print(x_train.shape)

In [ ]:
num_classes=10
y_train = k.utils.to_categorical(Y, num_classes)

In [ ]:
reg=None
num_filters=32
ac='relu'
adm=Adam(lr=0.001,decay=0, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
opt=adm
drop_dense=0.5
drop_conv=0

model = Sequential()

model.add(Conv2D(num_filters, (3, 3), activation=ac, kernel_regularizer=reg, input_shape=(32, 32,1),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 16x16x3xnum_filters
model.add(Dropout(0.2))

model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 8x8x3x(2*num_filters)
model.add(Dropout(0.3))

model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 4x4x3x(4*num_filters)
model.add(Dropout(0.4))

model.add(Conv2D(8*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(8*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 4x4x3x(8*num_filters)
model.add(Dropout(0.5))

model.add(Conv2D(16*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(16*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 4x4x3x(8*num_filters)
model.add(Dropout(0.6))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(512, activation=ac,kernel_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(drop_dense))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)


In [ ]:
model.summary()

In [ ]:
# import keras
# from keras.models import Sequential
# from keras.utils import np_utils
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D
# from keras import regularizers
# from keras.callbacks import LearningRateScheduler
# import numpy as np
# num_classes = 10
# weight_decay = 1e-4
# model = Sequential()
# model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
 
# model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.3))
 
# model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))
 
# model.add(Flatten())
# model.add(Dense(num_classes, activation='softmax'))
 
# model.summary()
 
# #data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     )
# datagen.fit(x_train)
 
# #training
# batch_size = 64
 
# opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
# model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
# model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
#                     steps_per_epoch=x_train.shape[0] ,epochs=125,
#                     verbose=1)

In [ ]:
history=model.fit(x_train, y_train, batch_size=128, epochs=100)

In [ ]:
data = pd.read_csv('../input/Test.csv')
data.head()

In [ ]:
interpolate(data)

In [ ]:
columns = [ "sparse_"+str(x) for x in range(0,1024) ]
test = np.array(data[columns])
from sklearn import preprocessing
test = preprocessing.StandardScaler().fit_transform(test)
x_test = test.reshape(test.shape[0], 32, 32,1)
print('x_train shape:', x_test.shape)
print(x_test.shape[0], 'train samples')
x_test=x_test.astype("float32")  

In [ ]:
x_test

In [ ]:
model_test_acc=model.predict_classes(x_test)
model_test_acc
print(model_test_acc)

In [ ]:
data_to_submit = pd.DataFrame({
    'id':np.arange(0, x_test.shape[0], 1),
    'label': model_test_acc.astype(int)
})

In [ ]:
data_to_submit.to_csv('NewLayers_Submit.csv', index = False)